In [135]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate


def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, omega, Pc, Tb, Tc, NCParams, Ant1 = None, Ant2 = None, Ant3 = None): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant1 = Ant1
        self.Ant2 = Ant2
        self.Ant3 = Ant3
    
    def Ant(self, T): # faixas de antoine, Escolhe uma
        if T > self.Ant1[3] and T < self.Ant1[4]:
            return self.Ant1[:3]

        if T > self.Ant2[3] and T < self.Ant2[4]:
            return self.Ant2[:3]

        if T > self.Ant3[3] and T < self.Ant3[4]:
            return self.Ant3[:3]

    def Antoine(self, T): # NIST, T em K
        Ant = self.Ant(T)
        AntA = Ant[0]
        AntB = Ant[1]
        AntC = Ant[2]
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura
        Ant = self.Ant1
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        Ant = self.Ant(T) # roda novamente para escolher a faixa correta de antoine
        return (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]

    def Estado(self, T, P): # Retorna o estado do composto
        if (T > self.Tc and P > self.Pc) or self.Ant1 == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if  P < self.Antoine(T):
                return "vapour"
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))
    
    def Cp_L(self, T):
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / self.Tc )
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / self.Tc)
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3)
        Z = np.roots([1, alpha, beta, gamma])
        Z = Z[np.isreal(Z)]
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self, T): # Por Riedel p147 V.Ness
        return R * T * ((1.092 * (np.log(self.Pc - 1.013)))/ (0.930 - (T / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(P, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        return integrate.quad(self.Cp_ig, Tpad, T)[0] + self.Hres(P, T)
                    case "liquid":
                        Tvap = self.TAntoine(P)
                        HGas = integrate.quad(self.Cp_ig, Tpad, Tvap)[0] + self.Hres(P, Tvap)
                        Hv = self.Hvap(Tvap)
                        Hl = integrate.quad(self.Cp_L, Tvap, T)
                        return HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
            case "liquid":
                match self.Estado(P, T):
                    case "gas" | "vapour":
                        Tvap = self.TAntoine(P)
                        Hl = integrate.quad(self.Cp_L(), Tpad, Tvap)
                        Hv = self.Hvap(Tvap)
                        HGas = integrate.quad(self.Cp_ig, Tvap, T)[0] + self.Hres(P, T)
                        return Hl + Hv + HGas
                    case "liquid":
                        return integrate.quad(self.Cp_L(), Tpad, T)

H2 = Comp( -0.217, # Prausnitz
          12.93, #Bar
          20.27, #K
          32.98, #K
         [3.249,
         0.422e3,
         0,
         0.083e-5])

CO = Comp(
        0.045, # Prausnitz
        34.94, #Bar
        81.66, #K
        132.85, #K
        [ 3.376,
         0.557e3,
         0,
         -0.031e-5])

CO2 = Comp(
        0.225, # Prausnitz
        73.74, #Bar
        194.69, #K
        304.12, #K
        [ 5.457,
        1.045e3,
        0,
        -1.157e-5]) 

N2 = Comp(
        0.037,
        33.98,
        77.35,
        126.20,
        [3.280,
        0.593e3,
        0,
        0.040e-5])
    
CH3OH = Comp(
        0.565,
        80.97,
        337.69,
        512.64,
        [2.211, # A
        12.216e3, # B
        -3.450e6, # C
        0], # D
        [5.15853, 1569.613, -34.846, 353.5, 512.63], # A, B, C, D, Tmin, Tmax Ambrose 1975
        [5.20409, 1581.341, -33.50, 288.1, 356.83], # Ambrose 1970
        [5.31301, 1676.569, -21.728, 353, 483]) # Hirata 1967

H2O = Comp(
        0.344, # Prausnitz
        220.64, #Bar
        373.15, #K
        647.14, #K
        [3.470, # V.Ness - |
        1.450e3,
        0,
        0.121e-5],
        [3.55959, 643.748, -198.043, 373, 573], # A, B, C, D, Tmin, Tmax do NIST liu e lindsey
        [4.6543, 1435.264, -64.848, 255.9, 373] # Stull, 1947
        )


In [136]:
# Para descoberta da temperatura Corrente 3



3363602.7302473076
181141744.29910716
177778141.56885985
177778141.56885985
